In [1]:
import pandas as pd
from datetime import datetime
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

from src.paths import *
from src.data_split import *

In [2]:
df = pd.read_parquet(path = TRANSFORMED_PATH + "/features.parquet")
df['target_rides_next_hour'] = pd.read_parquet(path = TRANSFORMED_PATH + "/target.parquet")
df

,rides_previous_672_hours,rides_previous_671_hours,rides_previous_670_hours,rides_previous_669_hours,rides_previous_668_hours,rides_previous_667_hours,rides_previous_666_hours,rides_previous_665_hours,rides_previous_664_hours,rides_previous_663_hours,...,rides_previous_7_hours,rides_previous_6_hours,rides_previous_5_hours,rides_previous_4_hours,rides_previous_3_hours,rides_previous_2_hours,rides_previous_1_hours,pickup_hour,pickup_location_id,target_rides_next_hour
0,1,1,0,2,0,0,1,2,1,5,...,0,0,0,0,0,0,0,2022-01-29 04:00:00,1,0
1,0,4,1,2,1,2,0,1,1,3,...,1,0,0,0,0,0,0,2022-01-30 04:00:00,1,0
2,0,0,0,2,0,0,0,0,2,1,...,0,0,0,0,0,0,0,2022-01-31 04:00:00,1,1
3,0,0,0,0,1,1,0,0,0,2,...,1,0,0,0,0,0,0,2022-02-01 04:00:00,1,0
4,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,2022-02-02 04:00:00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72307,3,4,1,1,1,0,1,2,2,3,...,29,15,4,12,7,11,4,2022-10-27 00:00:00,265,12
72308,9,4,3,3,2,4,2,6,1,3,...,29,17,12,4,5,8,9,2022-10-28 00:00:00,265,3
72309,7,6,3,4,3,5,7,6,5,10,...,29,13,9,10,5,10,7,2022-10-29 00:00:00,265,6
72310,6,5,8,6,6,0,1,2,8,6,...,8,10,7,3,3,6,2,2022-10-30 00:00:00,265,10


In [3]:
# Train - Test split

cutoff_dt = datetime(year=2022, month=6, day=1, hour=0, minute=0, second=0)
X_train,y_train,X_test,y_test = train_test_split(df,cutoff_dt)

print("Training data : ",X_train.shape,y_train.shape)
print("Test data : ",X_test.shape,y_test.shape)

Training data :  (32284, 674) (32284,)
Test data :  (40028, 674) (40028,)


### XGBoost model

- How to predict demand for the next hour?
    - Look at the no.of rides historically
- Pickup location is of categorical nature (despite being numerical) because it shows categories of locations

In [4]:
xgb_reg = xgb.XGBRegressor()

# Train model
xgb_reg.fit(X_train.drop(['pickup_hour','pickup_location_id'],axis=1),y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [5]:
y_pred_xgb = xgb_reg.predict(X_test.drop(['pickup_hour','pickup_location_id'],axis=1))

In [6]:
#Evaluate model

error_metric_xgb = mean_absolute_error(y_test,y_pred_xgb)
print("{:.4f}".format(error_metric_xgb))

2.6192
